In [11]:
!rm -rf /kaggle/working/ocr_trainer

In [13]:
%cd /kaggle/working

/kaggle/working


In [14]:
!git clone https://github.com/stephen18961/ocr_trainer.git

Cloning into 'ocr_trainer'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 73 (delta 4), reused 73 (delta 4), pack-reused 0
Unpacking objects: 100% (73/73), 13.61 MiB | 7.95 MiB/s, done.


In [7]:
%cd /kaggle/working/ocr_trainer/trainer

/kaggle/working/ocr_trainer/trainer


In [20]:
!pip install natsort

In [21]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [22]:
cudnn.benchmark = True
cudnn.deterministic = False

In [23]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [31]:
!cp /kaggle/input/tps-resnet-bilstm-attn.pth/pytorch/tps-resnet-bilstm-attn.pth/1/TPS-ResNet-BiLSTM-Attn.pth /kaggle/working/ocr_trainer/trainer

In [15]:
!mkdir all_data
!cd all_data && mkdir en_sample

In [13]:
!pwd

/kaggle/working/ocr_trainer/trainer


In [16]:
!cp /kaggle/input/plates2/trdg_results/* /kaggle/working/ocr_trainer/trainer/all_data/en_sample

In [1]:
!rm -rf /kaggle/working/ocr_trainer/trainer/all_data

In [27]:
#/kaggle/working/ocr_trainer/trainer/all_data/en_sample

# Training

In [24]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/en_sample
opt.select_data: ['all_data/en_sample']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/en_sample	 dataset: all_data/en_sample
all_data/en_sample/
sub-directory:	/.	 num samples: 0
num total samples of all_data/en_sample: 0 x 1.0 (total_data_usage_ratio) = 0
num samples of all_data/en_sample per batch: 32 x 1.0 (batch_ratio) = 32


ValueError: num_samples should be a positive integer value, but got num_samples=0